In [ ]:
from ultralytics import YOLO
import os
os.environ['WANDB_DISABLED'] = 'true'

In [ ]:
def train_yolo_model(model, data_yaml, epochs, imgsz, batch, save_period, name):
    results = model.train(
        data=data_yaml,
        epochs=epochs,
        imgsz=imgsz,
        batch=batch,
        save_period=save_period,
        name=name,
        verbose=False,
        exist_ok=True
    )
    return results

In [ ]:
model_a = YOLO("yolo11s.pt")
data_yaml_path = "soict.yaml"
# model_b = YOLO("yolov8s.pt")
# model_c = YOLO("yolov5s.pt")

In [ ]:
model_a = train_yolo_model(model_a, data_yaml_path, epochs = 20, imgsz = 640, batch=32, save_period = 1, name='trainning')
# model_b = train_yolo_model(model_b, data_yaml_path, epochs = 20, imgsz = 640, batch=32, save_period = 1, project='/kaggle/working', name='trainning_b')
# model_c = train_yolo_model(model_c, data_yaml_path, epochs = 20, imgsz = 640, batch=32, save_period = 1, project='/kaggle/working', name='trainning_c')


In [ ]:
model_a = r'trainning\weights\best.pt'
# model_b = '/kaggle/working/trainning_b/weights/best.pt'
# model_c = '/kaggle/working/trainning_b/weights/best.pt'
models = [model_a]


In [ ]:
old_name = r"public test"
test_images_path = r"test"

try:
    os.rename(old_name, test_images_path)
    print(f"Successfully renamed '{old_name}' to '{test_images_path}'")
except OSError as e:
    print(f"Error renaming directory: {e}")

In [ ]:
# Thư mục để lưu file kết quả
output_folder = r'predictions'
os.makedirs(output_folder, exist_ok=True)

for i, model_name in enumerate(models):
    model = YOLO(model_name)
    predictions = []
    model_output_folder = os.path.join(output_folder, model_name)  
    os.makedirs(model_output_folder, exist_ok=True)

    results = model.predict(source=test_images_path, save_dir=model_output_folder) 

    output_file = os.path.join(model_output_folder, f'predict.txt') 
    with open(output_file, 'w') as f:
        for result in results:
            for *xyxy, conf, cls in result.boxes.data.tolist():
                x1, y1, x2, y2 = map(int, xyxy)
                confidence = float(conf)
                class_id = int(cls)

                width = result.orig_shape[1]
                height = result.orig_shape[0]

                x_center = (x1 + x2) / 2 / width
                y_center = (y1 + y2) / 2 / height
                width_normalized = (x2 - x1) / width
                height_normalized = (y2 - y1) / height

                f.write(f"{os.path.basename(result.path)} {class_id} {x_center:.6f} {y_center:.6f} {width_normalized:.6f} {height_normalized:.6f} {confidence:.6f}\n")

    print(f"Dự đoán của model {model_name} đã được lưu tại: {output_file}")